In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
import time
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and validation data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0005_19042021'
# Hyperparameters search date
HS_DATE = '19042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0005_19042021


## Hyperparameters seach analysis

In [3]:
results_files_ls = glob.glob(os.path.join(RESULTS_PATH, DATA_ID, 'XGB_{}'.format(HS_DATE), 'XGB_{}_*.json'.format(HS_DATE)))

print('Number of results files: {}'.format(len(results_files_ls)))

Number of results files: 18


In [4]:
# Load all the results and generates a pandas dataframe
results_ls = []
for results_file in results_files_ls:
    with open(results_file) as json_file:
        results_dict = json.load(json_file)
        
    dict_aux = {}
    dict_aux['params_ID'] = results_dict['id']
    for key, value in results_dict['parameters'].items():
        dict_aux['param_' + key] = value
    for key, value in results_dict['cv_results'].items():
        dict_aux['__'.join([key, 'mean'])] = np.mean(value)
        dict_aux['__'.join([key, 'std'])] = np.std(value)

    results_ls.append(dict_aux)
        
results_df = pd.DataFrame(results_ls)
results_df

,params_ID,param_booster,param_eta,param_gamma,param_lambda,param_max_depth,param_nthread,param_objective,param_seed,param_subsample,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Test_Fx_MAE_mean__mean,Test_Fx_MAE_mean__std,Test_Fx_MAE_std__mean,Test_Fx_MAE_std__std,Test_Fx_MSE_mean__mean,Test_Fx_MSE_mean__std,Test_Fx_MSE_std__mean,Test_Fx_MSE_std__std,Test_Fx_R2_mean__mean,Test_Fx_R2_mean__std,Test_Fx_R2_std__mean,Test_Fx_R2_std__std,Test_Fy_MAE_mean__mean,Test_Fy_MAE_mean__std,Test_Fy_MAE_std__mean,Test_Fy_MAE_std__std,Test_Fy_MSE_mean__mean,Test_Fy_MSE_mean__std,Test_Fy_MSE_std__mean,Test_Fy_MSE_std__std,Test_Fy_R2_mean__mean,Test_Fy_R2_mean__std,Test_Fy_R2_std__mean,Test_Fy_R2_std__std,Test_Fz_MAE_mean__mean,Test_Fz_MAE_mean__std,Test_Fz_MAE_std__mean,Test_Fz_MAE_std__std,Test_Fz_MSE_mean__mean,Test_Fz_MSE_mean__std,Test_Fz_MSE_std__mean,Test_Fz_MSE_std__std,Test_Fz_R2_mean__mean,Test_Fz_R2_mean__std,Test_Fz_R2_std__mean,Test_Fz_R2_std__std
0,P63LB1GSD7,gbtree,0.2,0.01,1.0,15,8,reg:squarederror,0,1.00,407.360964,7.571134,3.929873,0.295803,1.649186,0.178539,35.166832,5.004749,28.175675,7.174984,0.880481,0.016705,0.080710,0.010682,4.359995,1.050321,2.769038,1.000630,58.059231,28.185341,65.648273,33.923707,0.773871,0.071547,0.130074,0.057365,5.539042,0.383757,2.128325,0.407765,62.455117,8.003072,39.499846,9.401047,0.903713,0.014600,0.038891,0.019795,9.135363,0.661232,2.565510,0.339938,189.357899,26.744436,103.001402,22.880423,0.314785,0.061079,0.165634,0.019312,8.045571,0.388390,4.379696,0.381700,188.017160,22.946639,195.167502,32.351094,0.141455,0.073916,0.222901,0.038939,14.634289,0.930613,6.008883,0.778121,535.661219,37.497986,492.182347,55.388760,0.257017,0.067955,0.216929,0.108998
1,H67AXHLY06,gbtree,0.3,0.05,0.8,30,8,reg:squarederror,0,1.00,712.268577,33.324726,2.840873,0.520416,2.088371,0.314985,25.318418,8.756451,32.253344,16.151761,0.914137,0.022135,0.101526,0.034722,3.834240,1.362203,2.865190,1.059335,53.615998,31.819607,66.375297,32.745520,0.792843,0.079514,0.160707,0.044359,3.117479,1.260741,1.720119,0.614502,23.533447,14.794557,21.905091,12.319507,0.958212,0.025013,0.036150,0.016520,9.443974,0.686706,2.536485,0.253819,201.110101,27.294826,105.538318,23.753954,0.266800,0.064058,0.180192,0.024382,8.409960,0.409865,4.539365,0.352870,201.738047,25.534681,207.699643,36.866825,0.066510,0.071279,0.260368,0.063677,14.917142,0.772594,6.045794,0.651397,561.252150,36.653781,508.504844,62.225038,0.212599,0.052057,0.228067,0.091205
2,7T8QE34V4X,gbtree,0.3,0.05,1.0,30,8,reg:squarederror,0,0.75,624.958020,24.826554,2.636239,0.615589,1.805612,0.312047,20.690286,7.654565,22.926566,9.335936,0.927869,0.023936,0.077042,0.016949,3.539635,1.302609,2.581526,1.108669,45.205678,28.868606,55.107759,32.364831,0.811000,0.075645,0.163944,0.052588,3.030591,1.172436,1.578006,0.602306,23.119706,14.353755,21.975957,12.484869,0.960531,0.024565,0.034917,0.016054,9.304694,0.689449,2.490216,0.240271,194.330475,27.289184,101.828443,22.197700,0.290838,0.064583,0.175559,0.023170,8.295945,0.401797,4.464522,0.380635,196.188216,25.654423,201.287413,36.349750,0.090996,0.072403,0.248597,0.052953,14.798087,0.787244,6.055860,0.698169,554.863563,37.706478,508.801330,66.258032,

In [5]:
# Sum up the scores by force axis in only one sortable score
for subset in ['Train', 'Valid']:
    for loss in ['MAE', 'MSE', 'R2']:
        results_df[subset + '_' + loss] = results_df[[subset + '_' + force + '_' + loss + '_mean__mean' for force in ['Fx', 'Fy', 'Fz']]].mean(axis=1)

In [6]:
# Sort the dataframe by the most relevant score
results_df = results_df.sort_values(['Valid_R2'], ascending=False)
results_df

,params_ID,param_booster,param_eta,param_gamma,param_lambda,param_max_depth,param_nthread,param_objective,param_seed,param_subsample,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Test_Fx_MAE_mean__mean,Test_Fx_MAE_mean__std,Test_Fx_MAE_std__mean,Test_Fx_MAE_std__std,Test_Fx_MSE_mean__mean,Test_Fx_MSE_mean__std,Test_Fx_MSE_std__mean,Test_Fx_MSE_std__std,Test_Fx_R2_mean__mean,Test_Fx_R2_mean__std,Test_Fx_R2_std__mean,Test_Fx_R2_std__std,Test_Fy_MAE_mean__mean,Test_Fy_MAE_mean__std,Test_Fy_MAE_std__mean,Test_Fy_MAE_std__std,Test_Fy_MSE_mean__mean,Test_Fy_MSE_mean__std,Test_Fy_MSE_std__mean,Test_Fy_MSE_std__std,Test_Fy_R2_mean__mean,Test_Fy_R2_mean__std,Test_Fy_R2_std__mean,Test_Fy_R2_std__std,Test_Fz_MAE_mean__mean,Test_Fz_MAE_mean__std,Test_Fz_MAE_std__mean,Test_Fz_MAE_std__std,Test_Fz_MSE_mean__mean,Test_Fz_MSE_mean__std,Test_Fz_MSE_std__mean,Test_Fz_MSE_std__std,Test_Fz_R2_mean__mean,Test_Fz_R2_mean__std,Test_Fz_R2_std__mean,Test_Fz_R2_std__std,Train_MAE,Train_MSE,Train_R2,Test_MAE,Test_MSE,Test_R2
15,II2ZWODEDU,gbtree,0.3,0.01,1.0,6,8,reg:squarederror,0,0.75,132.427205,2.486171,6.793238,0.238745,2.099205,0.305912,98.712573,7.304213,60.525326,12.370605,0.665521,0.018673,0.078124,0.013107,6.584058,0.451156,3.830365,0.530589,115.542388,21.903064,116.655806,33.594616,0.515698,0.034932,0.128127,0.018096,10.261865,0.493881,3.678117,0.238681,231.736952,19.367401,168.811004,9.571463,0.669413,0.034322,0.076154,0.035179,9.089635,0.497091,2.647547,0.299751,181.362189,23.181433,103.555434,26.957335,0.355081,0.048472,0.128365,0.024754,7.912292,0.231514,4.521532,0.459106,178.108281,15.218504,187.573071,22.788785,0.227676,0.056320,0.179241,0.027514,14.299141,0.653274,5.743920,0.724106,479.134282,27.029994,443.765247,56.188805,0.330557,0.056992,0.174416,0.101298,7.879720,148.663971,0.616877,10.433689,279.534917,0.304438
5,WFC1L57OY7,gbtree,0.4,0.01,0.8,6,8,reg:squarederror,0,0.75,128.852457,4.482540,6.641679,0.274204,1.902068,0.273806,94.517723,8.138756,52.820229,11.399821,0.673734,0.025379,0.082815,0.014188,6.585610,0.650551,3.907084,0.793151,119.674487,34.875506,125.134847,53.686170,0.515775,0.056879,0.144800,0.018130,10.084164,0.574321,3.596153,0.236421,223.374698,20.589200,159.871132,13.836090,0.676088,0.040836,0.082241,0.031188,9.090498,0.496375,2.583047,0.295293,181.771776,22.963477,101.914846,27.225507,0.350174,0.051087,0.131658,0.029234,7.903189,0.266383,4.516694,0.441777,178.483862,15.312320,188.698256,22.990408,0.226248,0.058746,0.179775,0.027372,14.285223,0.606498,5.706206,0.694397,474.895773,27.284464,442.082034,60.682194,0.335315,0.058059,0.171950,0.098401,7.770485,145.855636,0.621866,10.426303,278.383804,0.303912
4,SQ137IWT23,gbtree,0.4,0.10,0.8,6,8,reg:squarederror,0,0.50,105.037324,2.743210,6.836908,0.364652,2.071451,0.444640,100.583698,11.683508,59.923140,19.661538,0.655770,0.032064,0.099622,0.023449,6.479819,0.453406,3.714400,0.464092,112.177116,22.064625,114.086341,34.775933,0.519527,0.043509,0.144558,0.028825,9.971180,0.393512,3.570111,0.163951,221.025561,14.212194,164.128499,12.176379,0.682718,0.029614,0.077691,0.027732,9.110351,0.495971,2.617180,0.306478,181.719816,23.090115,102.551094,25.662146,0.349562,0.053085,0.136338,0.031301,7.882

In [7]:
best_params = dict(results_df.iloc[0][[col for col in results_df.columns if 'param_' in col]])
best_params = {key.replace('param_', ''): value for key, value in best_params.items()}
print('Best parameters: {}'.format(best_params))

Best parameters: {'booster': 'gbtree', 'eta': 0.3, 'gamma': 0.01, 'lambda': 1.0, 'max_depth': 6, 'nthread': 8, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}


## Best model

In [8]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

In [9]:
results = defaultdict(list)
tr_time = []
for target in range(Y_train.shape[1]):

    dtrain = xgb.DMatrix(data=X_train, label=Y_train[:, target])
    dtest = xgb.DMatrix(data=X_test, label=Y_test[:, target])

    callbacks = [xgb.callback.EarlyStopping(rounds=5, metric_name='rmse', maximize=False, save_best=True)]
    
    t_start = time.time()
    model = xgb.train(best_params, dtrain, evals=[(dtest, 'rmse')], callbacks=callbacks, verbose_eval=False)
    tr_time.append(time.time() - t_start)
    
    # Save the model
    model.save_model(os.path.join(RESULTS_PATH, DATA_ID, 'XGB_{}'.format(HS_DATE), 'XGB_best_model_{}_{}_{}.joblib'.format(target, HS_DATE, DATA_ID)))
    
    train_preds = model.predict(dtrain)
    test_preds = model.predict(dtest)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))

print('Training time: {:.4f}'.format(sum(tr_time)))

Training time: 320.8449


In [10]:
# # Display the score for each axis of each force cell
# for subset in ['Train', 'Test']:
#     for f, force in enumerate(['Fx', 'Fy', 'Fz']):
#         for c in range(N_CELLS):
#             for loss in ['MAE', 'MSE', 'R2']:
#                 scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
#                 print('{} {}{}{} {}: {:.4f}'.format(subset, force[0], c + 1, force[-1], loss, scores[c]))
# print('\n')

# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

Train Fx MAE: 7.1869 ± 1.6493
Train Fx MSE: 107.9562 ± 46.7758
Train Fx R2: 0.6132 ± 0.0684
Train Fy MAE: 6.6962 ± 3.5113
Train Fy MSE: 110.3752 ± 95.1319
Train Fy R2: 0.4701 ± 0.1710
Train Fz MAE: 10.4374 ± 3.4805
Train Fz MSE: 239.5518 ± 179.0259
Train Fz R2: 0.6489 ± 0.0732
Test Fx MAE: 10.9776 ± 3.8296
Test Fx MSE: 260.9678 ± 172.2369
Test Fx R2: 0.2024 ± 0.1711
Test Fy MAE: 9.8136 ± 6.3831
Test Fy MSE: 322.7382 ± 396.1981
Test Fy R2: 0.0787 ± 0.1623
Test Fz MAE: 16.6790 ± 7.2368
Test Fz MSE: 666.6956 ± 668.7742
Test Fz R2: 0.1651 ± 0.1559


In [ ]:
# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:, 3], Y_train[:, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:, 3], train_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:100, 3], Y_train[:100, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:100, 3], train_preds[:100, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_test[:, 3], Y_test[:, 4], label='true', alpha=0.3)
# plt.scatter(test_preds[:, 3], test_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()